<a href="https://colab.research.google.com/github/mrm8488/shared_colab_notebooks/blob/master/HF_SetFit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SetFit for Text Classification

In [1]:
! nvidia-smi

Mon Sep 26 18:46:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install -q transformers datasets setfit

     |████████████████████████████████| 4.9 MB 12.9 MB/s 
     |████████████████████████████████| 431 kB 58.4 MB/s 
     |████████████████████████████████| 120 kB 69.2 MB/s 
     |████████████████████████████████| 6.6 MB 63.9 MB/s 
     |████████████████████████████████| 115 kB 68.6 MB/s 
     |████████████████████████████████| 212 kB 54.7 MB/s 
     |████████████████████████████████| 127 kB 59.4 MB/s 
     |████████████████████████████████| 69 kB 8.6 MB/s 
     |████████████████████████████████| 362 kB 48.3 MB/s 
     |████████████████████████████████| 85 kB 1.8 MB/s 
     |████████████████████████████████| 1.3 MB 60.4 MB/s 


In [3]:
! huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [4]:
! git config --global credential.helper store

In [5]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [6]:
dataset = load_dataset("SetFit/SentEval-CR")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/SetFit___json/SetFit--SentEval-CR-3d6d995d44023096/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 3012
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 753
    })
})

In [8]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(dataset["train"])

,text,label,label_text
0,video output stopped working .,0,negative
1,. images taken indoor were n 't so clear .,0,negative
2,this thing seriously is a good player though .,1,positive
3,"if anyone is thinking of getting an ipod mini , forget it and go with this player .",1,positive
4,have used it frequently and have had no problems .,1,positive
5,the daylight pictures are brilliant and the indoor shots are very good provided that your subject is no more than about 8 feet away .,1,positive
6,"however , the battery life is good enough you should n 't be overly concerned with turning this phone off .",1,positive
7,"maybe this new design is more durable than the last , but i kind of doubt it .",0,negative
8,size - extremely small and convenient will fit in your pocket and not bother you .,1,positive
9,you also need to go in and manually remove many symantec files from the registry .,0,negative


In [10]:
# Select N examples per class (8 in this case)
train_ds = dataset["train"].shuffle(seed=42).select(range(8 * 2))
test_ds = dataset["test"]

In [11]:
show_random_elements(train_ds)

,text,label,label_text
0,"it fits into a hand well , it has a removable battery ( this is important ) , great sound quality , fm stereo , recorder , smooth ui , and a feature that most uni pods lack . . . char ! .",1,positive
1,"for a price that 's still less than even the lowest level ipod i was able to get this 40gb monster , and the best part is it works as great as it was advertised to and then some .",1,positive
2,i bought the player this week and i like it by far .,1,positive
3,neither message was answered ( they ask for 24 hours before replying - i 've been waiting 27 days . ),0,negative
4,the thought of not having to buy refills and just using regular bags is awesome ! .,1,positive
5,"as for bluetooth , no problems at all .",1,positive
6,the pc-side software can be goofy and takes a little getting used to but it 's not as bad as some reviewers have indicated .,0,negative
7,symantec should give us all a refund and apology ! .,0,negative
8,the day finally arrived when i was sure i 'd leave sprint .,0,negative
9,only problem is that is a bit heavy .,0,negative


In [12]:
# Load SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_epochs=20,
)

Downloading:   0%|          | 0.00/594 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights.


In [13]:
# Train and evaluate!
trainer.train()

16 train samples in total, 40 train steps with batch size 16


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

In [14]:
trainer.evaluate()

{'accuracy': 0.8193891102257637}

In [15]:
! git config --global user.email "your@email.com"

In [ ]:
trainer.push_to_hub("setfit-mpnet-base-v2-finetuned-sentEval-CR")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:42: FutureWarning: Deprecated positional argument(s) used in 'push_to_hub': pass repo_path_or_name='setfit-mpnet-base-v2-finetuned-sentEval-CR', repo_url=None, commit_message='Add model', organization=None, private=None, api_endpoint=None, use_auth_token=None, git_user=None, git_email=None, config=None, skip_lfs_files=False as keyword args. From version 0.12 passing these as positional arguments will result in an error,
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:92: FutureWarning: Deprecated argument(s) used in 'push_to_hub': repo_path_or_name, repo_url, organization, use_auth_token, git_user, git_email, skip_lfs_files. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Cloning https://huggingface.co/mrm8488/setfit-mpnet-base-v2-finetuned-sentEval-CR into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.34k/418M [00:00<?, ?B/s]

Upload file model_head.pkl:  49%|####8     | 3.34k/6.88k [00:00<?, ?B/s]